In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import plotly.graph_objects as go

# Generate example data (replace this with your real data)
np.random.seed(42)
data = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=100),
    'Close': np.random.rand(100) * 100,
})

# Function to calculate Moving Averages and RSI
def calculate_features(data, window_size=10):
    data['MA'] = data['Close'].rolling(window=window_size).mean()
    data['RSI'] = calculate_rsi(data['Close'])
    return data.dropna()

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(data, window_size=14):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window_size).mean()
    avg_loss = loss.rolling(window=window_size).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Function to create sequences for RNN
def create_sequences(data, sequence_length=10):
    sequences, labels = [], []

    for i in range(len(data) - sequence_length):
        sequence = data.iloc[i:i + sequence_length]['Close'].values
        label = data.iloc[i + sequence_length]['Close']
        sequences.append(sequence)
        labels.append(label)

    return np.array(sequences), np.array(labels)

# Preprocess the data
data = calculate_features(data)
data.dropna(inplace=True)

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Close', 'MA', 'RSI']])

data[['Close', 'MA', 'RSI']] = scaled_data

# Create sequences for RNN
sequence_length = 10
X, y = create_sequences(data, sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the RNN model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Make predictions on the test set
predictions = model.predict(X_test)

# Inverse transform the predictions and actual values to the original scale
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Plot the predictions and actual values
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=data['Date'].iloc[-len(predictions):], y=predictions.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='Stock Price Prediction with RNN',
                  xaxis_title='Date',
                  yaxis_title='Normalized Price')
fig.show()


Epoch 1/50


2023-11-14 21:05:45.092107: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 1s 6ms/step - loss: 0.2695
Epoch 2/50
2/2 [==============================] - 0s 3ms/step - loss: 0.2451
Epoch 3/50
2/2 [==============================] - 0s 4ms/step - loss: 0.2227
Epoch 4/50
2/2 [==============================] - 0s 3ms/step - loss: 0.2016
Epoch 5/50
2/2 [==============================] - 0s 4ms/step - loss: 0.1819
Epoch 6/50
2/2 [==============================] - 0s 3ms/step - loss: 0.1650
Epoch 7/50
2/2 [==============================] - 0s 3ms/step - loss: 0.1479
Epoch 8/50
2/2 [==============================] - 0s 3ms/step - loss: 0.1340
Epoch 9/50
2/2 [==============================] - 0s 2ms/step - loss: 0.1212
Epoch 10/50
2/2 [==============================] - 0s 3ms/step - loss: 0.1115
Epoch 11/50
2/2 [==============================] - 0s 2ms/step - loss: 0.1022
Epoch 12/50
2/2 [==============================] - 0s 3ms/step - loss: 0.0992
Epoch 13/50
2/2 [==============================] - 0s 3ms/step - loss: 0.1000
Epoch 

ValueError: non-broadcastable output operand with shape (16,1) doesn't match the broadcast shape (16,3)